<a href="https://colab.research.google.com/github/Tallaammar/W2023/blob/main/Thesis_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile
!pip install vaderSentiment
!pip install textblob
!pip install nltk

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Supplier_Internal_Records.csv to Supplier_Internal_Records.csv


In [5]:

import rarfile
import os

# Define the path to the .rar file
rar_file_path = '/content/Quotations.rar'

# Define the directory to extract to
extraction_path = '/content/Quotations'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Extract the .rar file
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(extraction_path)

# List the extracted files
extracted_files = os.listdir(extraction_path)
print("Extracted files:", extracted_files)

Extracted files: ['Quotation3.txt', 'Quotation2.txt', 'Quotation1.txt']


In [33]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import nltk

def sentiment_rankings(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Initialize VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Initialize dictionaries to store sentiment scores for each supplier
    supplier_sentiments1 = {supplier: 0 for supplier in df['Supplier'].unique()}
    supplier_sentiments2 = {supplier: 0 for supplier in df['Supplier'].unique()}
    supplier_sentiments3 = {supplier: 0 for supplier in df['Supplier'].unique()}

    # Perform sentiment analysis for each supplier using different methods
    for index, row in df.iterrows():
        feedback = row['Feedback']
        supplier = row['Supplier']

        # Using VADER sentiment analyzer
        sentiment_score1 = analyzer.polarity_scores(feedback)['compound']
        supplier_sentiments1[supplier] += sentiment_score1

        # Using TextBlob sentiment analyzer
        sentiment_score2 = TextBlob(feedback).sentiment.polarity
        supplier_sentiments2[supplier] += sentiment_score2

        # Using NLTK's VADER sentiment analyzer
        sentiment_score3 = analyzer.polarity_scores(feedback)['compound']
        supplier_sentiments3[supplier] += sentiment_score3

    # Calculate average sentiment scores
    sentiment_rankings = {}
    for key in supplier_sentiments1.keys():
        sentiment_rankings[key] = (supplier_sentiments1[key] + supplier_sentiments2[key] + supplier_sentiments3[key]) / 3

    # Sort the sentiment_rankings dictionary based on sentiment scores
    sorted_sentiment_rankings = sorted(sentiment_rankings.items(), key=lambda x: x[1], reverse=True)

    # Append ranking numbers to each tuple in the sorted_sentiment_rankings list
    for i, (supplier, score) in enumerate(sorted_sentiment_rankings, start=1):
        sorted_sentiment_rankings[i - 1] = (supplier, score, i)

    # Convert the sorted_sentiment_rankings list back to a dictionary
    sentiment_rankings = {supplier: (score, rank) for supplier, score, rank in sorted_sentiment_rankings}

    return sentiment_rankings

# Call the function with the file path
sentiment_rankings1 = sentiment_rankings("/content/Supplier_Internal_Records.csv")
print(sentiment_rankings1)


{'Supplier2': (2.236511111111111, 1), 'Supplier3': (2.042, 2), 'Supplier1': (0.40577777777777774, 3)}


In [15]:
import os
import re

# Define the folder path containing the text files
folder_path = '/content/Quotations'

# List all files in the folder
file_names = os.listdir(folder_path)

# Function to extract product information from a line
def extract_product_info(line):
    # Regular expression pattern to match product name
    product_pattern = re.compile(r'^\d+\.\s*(.*?):$')
    # Check if the line contains a product name
    match = product_pattern.match(line)
    if match:
        return match.group(1).strip(), {}
    else:
        return None, None

# Function to extract key-value pairs from a line
def extract_key_value(line):
    # Split the line to extract the keyword and its value
    parts = line.split(':', 1)
    if len(parts) == 2:
        return parts[0].strip(), parts[1].strip()
    else:
        return None, None

# Dictionary to store product information for each file
file_product_info = {}

# Iterate over each file
for file_name in file_names:
    # Open the current file
    with open(os.path.join(folder_path, file_name), 'r') as file:
        # Initialize variables to store information for products
        product_info = {}
        current_product = None

        # Read the file line by line
        for line in file:
            # Extract product information if available
            product_name, product_details = extract_product_info(line)
            if product_name:
                current_product = product_name
                product_info[current_product] = product_details
                continue

            # Extract key-value pairs for the current product
            if current_product:
                keyword, value = extract_key_value(line)
                if keyword and value:
                    product_info[current_product][keyword] = value

        # Store the product information for the current file
        file_product_info[file_name] = product_info

# Print all the product information for each file
for file_name, product_info in file_product_info.items():
    print(f"FILENAME: {file_name}\n")
    for product, details in product_info.items():
        print(f"{product}:")
        for keyword, value in details.items():
            print(f"- {keyword}: {value}")
        print()
    print("-" * 50 + "\n")


FILENAME: Quotation3.txt

Cassette:
- - Price per unit: $4.50
- - Delivery Time: 8 days
- - Discount: 6% off for orders exceeding 900
- - Payment: Flexible payment terms with a 30-day credit option.
- - Warranty: 9-month warranty with replacement service.
- - Guarantees: Quality assurance guaranteed.

Gears:
- - Price per unit: $2
- - Delivery Time: 14 days
- - Discount: 5% off for additional quantities after 4,000 units.
- - Payment: Flexible payment terms with a 30-day credit option.
- - Warranty: 6-month warranty with replacement service.
- - Guarantees: Performance satisfaction guaranteed.

--------------------------------------------------

FILENAME: Quotation2.txt

Cassette:
- - Price per unit: $5.20
- - Delivery Time: 10 days
- - Discount: 3% off on orders exceeding 500
- - Payment: within 10 days from the date of delivery.
- - Warranty: 6-month warranty with next-business-day replacement guarantee.
- - Guarantees: Product performance guarantee provided.

Gears:
- - Price per un

In [6]:
import os
import re

# Define the folder path containing the text files
folder_path = '/content/Quotations'

# List all files in the folder
file_names = os.listdir(folder_path)

# Function to extract product information from a line
def extract_product_info(line):
    # Regular expression pattern to match product name
    product_pattern = re.compile(r'^\d+\.\s*(.*?):$')
    # Check if the line contains a product name
    match = product_pattern.match(line)
    if match:
        return match.group(1).strip(), {}
    else:
        return None, None

# Function to extract key-value pairs from a line
def extract_key_value(line):
    # Split the line to extract the keyword and its value
    parts = line.split(':', 1)
    if len(parts) == 2:
        return parts[0].strip(), parts[1].strip()
    else:
        return None, None

# Dictionary to store product information for each file
file_product_info = {}

# Iterate over each file
for file_name in file_names:
    # Open the current file
    with open(os.path.join(folder_path, file_name), 'r') as file:
        # Initialize dictionary to store product information for this file
        product_info = {}
        current_product = None

        # Read the file line by line
        for line in file:
           # print(f"Reading line: {line.strip()}")  # Print the line being read

            # Extract product information if available
            product_name, product_details = extract_product_info(line)
            if product_name:
                #print(f"Extracted product: {product_name}")  # Print the extracted product name
                current_product = product_name
                product_info[current_product] = product_details
                continue

            # Extract key-value pairs for the current product
            if current_product:
                keyword, value = extract_key_value(line)
                if keyword and value:
                   # print(f"Extracted key-value pair: {keyword}: {value}")  # Print the extracted key-value pair
                    product_info[current_product][keyword] = value

        # Store product information for this file
        file_product_info[file_name] = product_info

# Print the dictionary structure
print(file_product_info)




{'Quotation3.txt': {'Cassette': {'- Price per unit': '$4.50', '- Delivery Time': '8 days', '- Discount': '6% off for orders exceeding 900', '- Payment': 'Flexible payment terms with a 30-day credit option.', '- Warranty': '9-month warranty with replacement service.', '- Guarantees': 'Quality assurance guaranteed.'}, 'Gears': {'- Price per unit': '$2', '- Delivery Time': '14 days', '- Discount': '5% off for additional quantities after 4,000 units.', '- Payment': 'Flexible payment terms with a 30-day credit option.', '- Warranty': '6-month warranty with replacement service.', '- Guarantees': 'Performance satisfaction guaranteed.'}}, 'Quotation2.txt': {'Cassette': {'- Price per unit': '$5.20', '- Delivery Time': '10 days', '- Discount': '3% off on orders exceeding 500', '- Payment': 'within 10 days from the date of delivery.', '- Warranty': '6-month warranty with next-business-day replacement guarantee.', '- Guarantees': 'Product performance guarantee provided.'}, 'Gears': {'- Price per u

In [34]:
import re

def calculate_cost(file_product_info, product_quantities):
    total_costs = {}
    for file_name, product_info in file_product_info.items():
        total_cost = 0
        product_costs = {}
        for product, details in product_info.items():
            price_per_unit_match = re.search(r'\$([\d.]+)', details.get('- Price per unit', ''))
            if price_per_unit_match:
                price_per_unit = float(price_per_unit_match.group(1))
                quantity = product_quantities.get(product, 0)
                product_cost = quantity * price_per_unit

                # Case 1: x% off orders above or exceeding y units
                discount_info = details.get('- Discount', '')
                #discount_percentage_match = re.search(r'(\d+)% off for orders exceeding (\d+)', discount_info)
                # Matching the discount percentage and threshold quantity for orders exceeding a certain amount
                discount_percentage_match = re.search(r'(\d+)% off (?:for|on) orders exceeding (\d+)', discount_info)

                if discount_percentage_match:
                    discount_percentage = float(discount_percentage_match.group(1))
                    threshold_quantity_str = discount_percentage_match.group(2)
                    # Check if threshold_quantity_str is not None before replacing commas
                    threshold_quantity = int(threshold_quantity_str.replace(',', '')) if threshold_quantity_str else 0
                    if quantity > threshold_quantity:
                        discounted_quantity = quantity - threshold_quantity
                        product_cost -= quantity * price_per_unit * (discount_percentage / 100)

                # Case 2: x% off for additional quantities after y units
                discount_additional_match = re.search(r'(\d+)% off for additional quantities (?:after|above) ([\d,]+)', discount_info)
                if discount_additional_match:
                    discount_percentage = float(discount_additional_match.group(1))
                    threshold_quantity_str = discount_additional_match.group(2)
                    # Check if threshold_quantity_str is not None before replacing commas
                    threshold_quantity = int(threshold_quantity_str.replace(',', '')) if threshold_quantity_str else 0
                    if quantity > threshold_quantity:
                        additional_quantity = quantity - threshold_quantity
                        product_cost = (threshold_quantity * price_per_unit) + ((price_per_unit * additional_quantity)- ((additional_quantity * price_per_unit) * (discount_percentage / 100)))
                        #print(additional_quantity, price_per_unit, discount_percentage, quantity, threshold_quantity)
                total_cost += product_cost
                product_costs[product] = product_cost
        total_costs[file_name] = {'Total Cost': total_cost, 'Product Costs': product_costs}
    return total_costs


# Example usage:
product_quantities = {'Cassette': 5000, 'Gears': 5000}  # Example with 5000 units for both products
total_costs = calculate_cost(file_product_info, product_quantities)


# Sorting based on total cost
cost_rankings1 = sorted(total_costs.items(), key=lambda x: x[1]['Total Cost'], reverse=True)

rank = 0  # Initialize rank outside the loop
previous_cost = None  # Initialize previous_cost variable

for i, (file_name, costs_info) in enumerate(cost_rankings1):
    cost = costs_info['Total Cost']
    if cost != previous_cost:
        rank = i + 1  # Increment rank only when encountering a new cost
    cost_rankings1[i] = (file_name, costs_info, rank)  # Append rank to tuple
    previous_cost = cost  # Update previous_cost for the next iteration

for file_name, costs_info, rank in cost_rankings1:
    cost = costs_info['Total Cost']
    print(f"{file_name}: ${cost:.2f}, Ranking: {rank}")


Quotation2.txt: $35220.00, Ranking: 1
Quotation1.txt: $31750.00, Ranking: 2
Quotation3.txt: $31050.00, Ranking: 3


In [35]:
print(cost_rankings1)

[('Quotation2.txt', {'Total Cost': 35220.0, 'Product Costs': {'Cassette': 25220.0, 'Gears': 10000.0}}, 1), ('Quotation1.txt', {'Total Cost': 31750.0, 'Product Costs': {'Cassette': 23750.0, 'Gears': 8000.0}}, 2), ('Quotation3.txt', {'Total Cost': 31050.0, 'Product Costs': {'Cassette': 21150.0, 'Gears': 9900.0}}, 3)]


In [49]:
import re

def calculate_cost(file_product_info, product_quantities):
    total_costs = {}
    for file_name, product_info in file_product_info.items():
        total_cost = 0
        product_costs = {}
        for product, details in product_info.items():
            price_per_unit_match = re.search(r'\$([\d.]+)', details.get('- Price per unit', ''))
            if price_per_unit_match:
                price_per_unit = float(price_per_unit_match.group(1))
                quantity = product_quantities.get(product, 0)
                product_cost = quantity * price_per_unit

                # Case 1: x% off orders above or exceeding y units
                discount_info = details.get('- Discount', '')
                #discount_percentage_match = re.search(r'(\d+)% off for orders exceeding (\d+)', discount_info)
                # Matching the discount percentage and threshold quantity for orders exceeding a certain amount
                discount_percentage_match = re.search(r'(\d+)% off (?:for|on) orders exceeding (\d+)', discount_info)

                if discount_percentage_match:
                    discount_percentage = float(discount_percentage_match.group(1))
                    threshold_quantity_str = discount_percentage_match.group(2)
                    # Check if threshold_quantity_str is not None before replacing commas
                    threshold_quantity = int(threshold_quantity_str.replace(',', '')) if threshold_quantity_str else 0
                    if quantity > threshold_quantity:
                        discounted_quantity = quantity - threshold_quantity
                        product_cost -= quantity * price_per_unit * (discount_percentage / 100)

                # Case 2: x% off for additional quantities after y units
                discount_additional_match = re.search(r'(\d+)% off for additional quantities (?:after|above) ([\d,]+)', discount_info)
                if discount_additional_match:
                    discount_percentage = float(discount_additional_match.group(1))
                    threshold_quantity_str = discount_additional_match.group(2)
                    # Check if threshold_quantity_str is not None before replacing commas
                    threshold_quantity = int(threshold_quantity_str.replace(',', '')) if threshold_quantity_str else 0
                    if quantity > threshold_quantity:
                        additional_quantity = quantity - threshold_quantity
                        product_cost = (threshold_quantity * price_per_unit) + ((price_per_unit * additional_quantity)- ((additional_quantity * price_per_unit) * (discount_percentage / 100)))
                        #print(additional_quantity, price_per_unit, discount_percentage, quantity, threshold_quantity)
                total_cost += product_cost
                product_costs[product] = product_cost
        total_costs[file_name] = {'Total Cost': total_cost, 'Product Costs': product_costs}
    return total_costs


def cost_rankings(file_product_info, product_quantities):
    total_costs = calculate_cost(file_product_info, product_quantities)

    # Sorting based on total cost
    cost_rankings = sorted(total_costs.items(), key=lambda x: x[1]['Total Cost'], reverse=True)

    rank = 0  # Initialize rank outside the loop
    previous_cost = None  # Initialize previous_cost variable

    for i, (file_name, costs_info) in enumerate(cost_rankings):
        cost = costs_info['Total Cost']
        if cost != previous_cost:
            rank = i + 1  # Increment rank only when encountering a new cost
        cost_rankings[i] = (file_name, costs_info, rank)  # Append rank to tuple
        previous_cost = cost  # Update previous_cost for the next iteration

    return cost_rankings

# Example usage:
product_quantities = {'Cassette': 5000, 'Gears': 5000}  # Example with 5000 units for both products


cost_rankings3= cost_rankings(file_product_info, product_quantities)
print(cost_rankings3)



[('Quotation2.txt', {'Total Cost': 35220.0, 'Product Costs': {'Cassette': 25220.0, 'Gears': 10000.0}}, 1), ('Quotation1.txt', {'Total Cost': 31750.0, 'Product Costs': {'Cassette': 23750.0, 'Gears': 8000.0}}, 2), ('Quotation3.txt', {'Total Cost': 31050.0, 'Product Costs': {'Cassette': 21150.0, 'Gears': 9900.0}}, 3)]


In [36]:
def extract_delivery_time(file_product_info):
    delivery_times = {}

    for file_name, products in file_product_info.items():
        max_delivery_time = 0
        for product_info in products.values():
            if '- Delivery Time' in product_info:
                delivery_time = int(product_info['- Delivery Time'].split()[0])
                max_delivery_time = max(max_delivery_time, delivery_time)
        delivery_times[file_name] = max_delivery_time

    return delivery_times


def delivery_ranking(file_product_info):
    delivery_times = extract_delivery_time(file_product_info)
    sorted_delivery_times = sorted(delivery_times.items(), key=lambda x: x[1])

    # Add ranking number considering ties
    ranked_quotations = []
    unique_delivery_times = {}
    current_rank = 0
    for quotation, delivery_time in sorted_delivery_times:
        if delivery_time not in unique_delivery_times:
            current_rank += 1
            unique_delivery_times[delivery_time] = current_rank
        ranked_quotations.append((quotation, delivery_time, unique_delivery_times[delivery_time]))

    return ranked_quotations


delivery_rankings1 = delivery_ranking(file_product_info)

for i, (quotation, delivery_time, ranking) in enumerate(delivery_rankings1, start=1):
    print(f"{i}) {quotation}, {delivery_time} days, Ranking: {ranking}")


1) Quotation2.txt, 10 days, Ranking: 1
2) Quotation3.txt, 14 days, Ranking: 2
3) Quotation1.txt, 15 days, Ranking: 3


In [51]:
print(delivery_rankings1)

[('Quotation2.txt', 10, 1), ('Quotation3.txt', 14, 2), ('Quotation1.txt', 15, 3)]


In [52]:
def extract_delivery_time(file_product_info):
    delivery_times = {}

    for file_name, products in file_product_info.items():
        max_delivery_time = 0
        for product_info in products.values():
            if '- Delivery Time' in product_info:
                delivery_time = int(product_info['- Delivery Time'].split()[0])
                max_delivery_time = max(max_delivery_time, delivery_time)
        delivery_times[file_name] = max_delivery_time

    return delivery_times


def delivery_rankings(file_product_info):
    delivery_times = extract_delivery_time(file_product_info)
    sorted_delivery_times = sorted(delivery_times.items(), key=lambda x: x[1])

    # Add ranking number considering ties
    ranked_quotations = []
    unique_delivery_times = {}
    current_rank = 0
    for quotation, delivery_time in sorted_delivery_times:
        if delivery_time not in unique_delivery_times:
            current_rank += 1
            unique_delivery_times[delivery_time] = current_rank
        ranked_quotations.append((quotation, delivery_time, unique_delivery_times[delivery_time]))

    return ranked_quotations

delivery_rankings3 = delivery_rankings(file_product_info)

print(delivery_rankings3)


[('Quotation2.txt', 10, 1), ('Quotation3.txt', 14, 2), ('Quotation1.txt', 15, 3)]


In [37]:
def extract_payment_terms(file_product_info):
    payment_terms = {}

    for file_name, products in file_product_info.items():
        max_payment_days = 0
        for product_info in products.values():
            if '- Payment' in product_info:
                payment_info = product_info['- Payment'].lower().split()
                if 'within' in payment_info:
                    days_index = payment_info.index('within') + 1
                    days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                    max_payment_days = max(max_payment_days, days)
                elif 'net' in payment_info:
                    days_index = payment_info.index('net') + 1
                    days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                    max_payment_days = max(max_payment_days, days)
                elif 'flexible' in payment_info:
                    for i, word in enumerate(payment_info):
                        if word == 'credit':
                            days_index = i - 1
                            days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                            max_payment_days = max(max_payment_days, days)
                            break
        payment_terms[file_name] = max_payment_days

    return payment_terms


def payment_ranking(file_product_info):
    payment_terms = extract_payment_terms(file_product_info)
    sorted_payment_terms = sorted(payment_terms.items(), key=lambda x: x[1], reverse=True)

    # Add ranking number considering ties
    ranked_payments = []
    unique_payment_terms = {}
    current_rank = 0
    for quotation, payment_days in sorted_payment_terms:
        if payment_days not in unique_payment_terms:
            current_rank += 1
            unique_payment_terms[payment_days] = current_rank
        ranked_payments.append((quotation, payment_days, unique_payment_terms[payment_days]))

    return ranked_payments

payment_rankings1 = payment_ranking(file_product_info)

for i, (quotation, payment_days, ranking) in enumerate(payment_rankings1, start=1):
    print(f"{i}) {quotation}, {payment_days} days, Ranking: {ranking}")


1) Quotation1.txt, 40 days, Ranking: 1
2) Quotation3.txt, 30 days, Ranking: 2
3) Quotation2.txt, 10 days, Ranking: 3


In [54]:
print(payment_rankings1)

[('Quotation1.txt', 40, 1), ('Quotation3.txt', 30, 2), ('Quotation2.txt', 10, 3)]


In [55]:
def extract_payment_terms(file_product_info):
    payment_terms = {}

    for file_name, products in file_product_info.items():
        max_payment_days = 0
        for product_info in products.values():
            if '- Payment' in product_info:
                payment_info = product_info['- Payment'].lower().split()
                if 'within' in payment_info:
                    days_index = payment_info.index('within') + 1
                    days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                    max_payment_days = max(max_payment_days, days)
                elif 'net' in payment_info:
                    days_index = payment_info.index('net') + 1
                    days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                    max_payment_days = max(max_payment_days, days)
                elif 'flexible' in payment_info:
                    for i, word in enumerate(payment_info):
                        if word == 'credit':
                            days_index = i - 1
                            days = int(''.join(filter(str.isdigit, payment_info[days_index])))
                            max_payment_days = max(max_payment_days, days)
                            break
        payment_terms[file_name] = max_payment_days

    return payment_terms


def payment_rankings(file_product_info):
    payment_terms = extract_payment_terms(file_product_info)
    sorted_payment_terms = sorted(payment_terms.items(), key=lambda x: x[1], reverse=True)

    # Add ranking number considering ties
    ranked_payments = []
    unique_payment_terms = {}
    current_rank = 0
    for quotation, payment_days in sorted_payment_terms:
        if payment_days not in unique_payment_terms:
            current_rank += 1
            unique_payment_terms[payment_days] = current_rank
        ranked_payments.append((quotation, payment_days, unique_payment_terms[payment_days]))

    return ranked_payments



payment_rankings3 = payment_rankings(file_product_info)

print(payment_rankings3)



[('Quotation1.txt', 40, 1), ('Quotation3.txt', 30, 2), ('Quotation2.txt', 10, 3)]


In [38]:
import re

def extract_warranty_info(file_product_info):
    warranties = {}

    for file_name, products in file_product_info.items():
        total_warranty_months = 0
        no_replacement_service = False
        free_replacement_service = False

        for product_info in products.values():
            if '- Warranty' in product_info:
                warranty_text = product_info['- Warranty'].lower()

                # Check for no replacement services
                if 'no replacement services' in warranty_text:
                    no_replacement_service = True
                    break

                # Check for free replacement service
                free_replacement_pattern = re.compile(r'\b(free replacement|replacement service|next-business-day replacement)\b', re.IGNORECASE)

                # Check for free replacement service using regex pattern
                free_replacement_service = bool(re.search(free_replacement_pattern, warranty_text))

                # Extract warranty time
                words = re.findall(r'\b\d+-?\s?month\b|\b\d+-?\s?year\b', warranty_text)
                for word in words:
                    number = int(re.search(r'\d+', word).group())
                    if 'year' in word:
                        total_warranty_months += number * 12
                    else:
                        total_warranty_months += number

        # Only consider quotations without "no replacement services"
        if not no_replacement_service:
            warranties[file_name] = (total_warranty_months, free_replacement_service)

    return warranties

def warranty_ranking(file_product_info):
    warranties = extract_warranty_info(file_product_info)
    sorted_warranties = sorted(warranties.items(), key=lambda x: (x[1][0], x[1][1]), reverse=True)
    rankings = []

    current_rank = 1
    current_warranty = None

    for i, (quotation, (total_months, free_replacement)) in enumerate(sorted_warranties, start=1):
        if (total_months, free_replacement) != current_warranty:
            current_rank = i
            current_warranty = (total_months, free_replacement)
        years, months = divmod(total_months, 12)
        rankings.append((current_rank, quotation, years, months, free_replacement))

    # Check if any quotations are missing from the rankings
    quoted_files = set([quotation for _, quotation, _, _, _ in rankings])
    missing_files = set(file_product_info.keys()) - quoted_files

    # Append missing files to the end of the rankings
    for file_name in missing_files:
        current_rank += 1
        rankings.append((current_rank, file_name, None, None, None))

    return rankings

warranty_rankings1 = warranty_ranking(file_product_info)

z=1
for rank, quotation, years, months, free_replacement in warranty_rankings1:
    warranty_info = f"{years} years, {months} months - {'Replacement Service' if free_replacement else 'No Replacement Service'}"
    print(f"{z}) {quotation}, {warranty_info}, Ranking: {rank}")
    z+=1


1) Quotation3.txt, 1 years, 3 months - Replacement Service, Ranking: 1
2) Quotation2.txt, 1 years, 0 months - Replacement Service, Ranking: 2
3) Quotation1.txt, None years, None months - No Replacement Service, Ranking: 3


In [57]:
print(warranty_rankings1)

[(1, 'Quotation3.txt', 1, 3, True), (2, 'Quotation2.txt', 1, 0, True), (3, 'Quotation1.txt', None, None, None)]


In [58]:
import re

def extract_warranty_info(file_product_info):
    warranties = {}

    for file_name, products in file_product_info.items():
        total_warranty_months = 0
        no_replacement_service = False
        free_replacement_service = False

        for product_info in products.values():
            if '- Warranty' in product_info:
                warranty_text = product_info['- Warranty'].lower()

                # Check for no replacement services
                if 'no replacement services' in warranty_text:
                    no_replacement_service = True
                    break

                # Check for free replacement service
                free_replacement_pattern = re.compile(r'\b(free replacement|replacement service|next-business-day replacement)\b', re.IGNORECASE)

                # Check for free replacement service using regex pattern
                free_replacement_service = bool(re.search(free_replacement_pattern, warranty_text))

                # Extract warranty time
                words = re.findall(r'\b\d+-?\s?month\b|\b\d+-?\s?year\b', warranty_text)
                for word in words:
                    number = int(re.search(r'\d+', word).group())
                    if 'year' in word:
                        total_warranty_months += number * 12
                    else:
                        total_warranty_months += number

        # Only consider quotations without "no replacement services"
        if not no_replacement_service:
            warranties[file_name] = (total_warranty_months, free_replacement_service)

    return warranties

def warranty_rankings(file_product_info):
    warranties = extract_warranty_info(file_product_info)
    sorted_warranties = sorted(warranties.items(), key=lambda x: (x[1][0], x[1][1]), reverse=True)
    rankings = []

    current_rank = 1
    current_warranty = None

    for i, (quotation, (total_months, free_replacement)) in enumerate(sorted_warranties, start=1):
        if (total_months, free_replacement) != current_warranty:
            current_rank = i
            current_warranty = (total_months, free_replacement)
        years, months = divmod(total_months, 12)
        rankings.append((current_rank, quotation, years if years != 0 else None, months if months != 0 else None, free_replacement))

    # Check if any quotations are missing from the rankings
    quoted_files = set([quotation for _, quotation, _, _, _ in rankings])
    missing_files = set(file_product_info.keys()) - quoted_files

    # Append missing files to the end of the rankings
    for file_name in missing_files:
        current_rank += 1
        rankings.append((current_rank, file_name, None, None, None))

    return rankings



warranty_rankings3 = warranty_rankings(file_product_info)

print(warranty_rankings3)


[(1, 'Quotation3.txt', 1, 3, True), (2, 'Quotation2.txt', 1, None, True), (3, 'Quotation1.txt', None, None, None)]


In [44]:
weights= [('Sentiment', 0.1), ('Cost', 0.5), ('Delivery', 0.2), ('Payment', 0.1), ('Warranty', 0.1)]

def recommendation(sentiment_rankings1, cost_rankings1, delivery_rankings1, payment_rankings1, warranty_rankings1, weights):


    score_dict = {}

    # Assign scores based on rankings and multiply by weights
    for weight in weights:
        category = weight[0]
        weight_value = weight[1]
        print("Processing category:", category)

        # Determine the number of quotations in the current category
        num_quotations = len(sentiment_rankings1) if category.lower() == 'sentiment' else \
                         len(cost_rankings1) if category.lower() == 'cost' else \
                         len(delivery_rankings1) if category.lower() == 'delivery' else \
                         len(payment_rankings1) if category.lower() == 'payment' else \
                         len(warranty_rankings1)

        # Assign scores based on rankings
        for i in range(1, num_quotations + 1):
            print(f"Calculating scores for {category} {i}...")
            if category.lower() == 'sentiment':
                supplier = f'Supplier{i}'
                sentiment_score, _ = sentiment_rankings1[supplier]
                score = weight_value * sentiment_score
            elif category.lower() == 'cost':
                quotation, _, ranking = cost_rankings1[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'delivery':
                quotation, _, ranking = delivery_rankings1[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'payment':
                quotation, _, ranking = payment_rankings1[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'warranty':
                _, quotation, _, _, warranty_bool = warranty_rankings1[i - 1]
                if not warranty_bool:
                    continue
                score = weight_value * (4 - ranking)

            print(f"Score for {quotation if category.lower() != 'sentiment' else supplier}: {score}")

            # Store the score in the dictionary
            if category.lower() == 'sentiment':
                score_dict[supplier] = score_dict.get(supplier, 0) + score
            else:
                score_dict[quotation] = score_dict.get(quotation, 0) + score

    # Find the quotation with the highest score
    best_quotation = max(score_dict, key=score_dict.get)

    return best_quotation

best_quotation = recommendation(sentiment_rankings1, cost_rankings1, delivery_rankings1, payment_rankings1, warranty_rankings1, weights)
print("Best quotation:", best_quotation)

Processing category: Sentiment
Calculating scores for Sentiment 1...
Score for Supplier1: 0.040577777777777775
Calculating scores for Sentiment 2...
Score for Supplier2: 0.22365111111111113
Calculating scores for Sentiment 3...
Score for Supplier3: 0.2042
Processing category: Cost
Calculating scores for Cost 1...
Score for Quotation2.txt: 1.5
Calculating scores for Cost 2...
Score for Quotation1.txt: 1.0
Calculating scores for Cost 3...
Score for Quotation3.txt: 0.5
Processing category: Delivery
Calculating scores for Delivery 1...
Score for Quotation2.txt: 0.6000000000000001
Calculating scores for Delivery 2...
Score for Quotation3.txt: 0.4
Calculating scores for Delivery 3...
Score for Quotation1.txt: 0.2
Processing category: Payment
Calculating scores for Payment 1...
Score for Quotation1.txt: 0.30000000000000004
Calculating scores for Payment 2...
Score for Quotation3.txt: 0.2
Calculating scores for Payment 3...
Score for Quotation2.txt: 0.1
Processing category: Warranty
Calculatin

In [59]:
weights2= [('Sentiment', 0.1), ('Cost', 0.5), ('Delivery', 0.2), ('Payment', 0.1), ('Warranty', 0.1)]

def recommendation(product_quantities, weights2, file_product_info):

    sentiment_rankings2= sentiment_rankings("/content/Supplier_Internal_Records.csv")
    cost_rankings2= cost_rankings(file_product_info, product_quantities)
    print(cost_rankings2)
    delivery_rankings2= delivery_rankings(file_product_info)
    print(delivery_rankings2)
    payment_rankings2= payment_rankings(file_product_info)
    warranty_rankings2=warranty_rankings(file_product_info)
    score_dict = {}

    # Assign scores based on rankings and multiply by weights
    for weight in weights2:
        category = weight[0]
        weight_value = weight[1]
        print("Processing category:", category)

        # Determine the number of quotations in the current category
        num_quotations = len(sentiment_rankings2) if category.lower() == 'sentiment' else \
                         len(cost_rankings2) if category.lower() == 'cost' else \
                         len(delivery_rankings2) if category.lower() == 'delivery' else \
                         len(payment_rankings2) if category.lower() == 'payment' else \
                         len(warranty_rankings2)

        # Assign scores based on rankings
        for i in range(1, num_quotations + 1):
            print(f"Calculating scores for {category} {i}...")
            if category.lower() == 'sentiment':
                supplier = f'Supplier{i}'
                sentiment_score, _ = sentiment_rankings2[supplier]
                score = weight_value * sentiment_score
            elif category.lower() == 'cost':
                quotation, _, ranking = cost_rankings2[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'delivery':
                quotation, _, ranking = delivery_rankings2[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'payment':
                quotation, _, ranking = payment_rankings2[i - 1]
                score = weight_value * (4 - ranking)
            elif category.lower() == 'warranty':
                _, quotation, _, _, warranty_bool = warranty_rankings2[i - 1]
                if not warranty_bool:
                    continue
                score = weight_value * (4 - ranking)

            print(f"Score for {quotation if category.lower() != 'sentiment' else supplier}: {score}")

            # Store the score in the dictionary
            if category.lower() == 'sentiment':
                score_dict[supplier] = score_dict.get(supplier, 0) + score
            else:
                score_dict[quotation] = score_dict.get(quotation, 0) + score

    # Find the quotation with the highest score
    best_quotation = max(score_dict, key=score_dict.get)

    return best_quotation

best_quotation = recommendation(product_quantities, weights2, file_product_info)
print("Best quotation:", best_quotation)

[('Quotation2.txt', {'Total Cost': 35220.0, 'Product Costs': {'Cassette': 25220.0, 'Gears': 10000.0}}, 1), ('Quotation1.txt', {'Total Cost': 31750.0, 'Product Costs': {'Cassette': 23750.0, 'Gears': 8000.0}}, 2), ('Quotation3.txt', {'Total Cost': 31050.0, 'Product Costs': {'Cassette': 21150.0, 'Gears': 9900.0}}, 3)]
[('Quotation2.txt', 10, 1), ('Quotation3.txt', 14, 2), ('Quotation1.txt', 15, 3)]
Processing category: Sentiment
Calculating scores for Sentiment 1...
Score for Supplier1: 0.040577777777777775
Calculating scores for Sentiment 2...
Score for Supplier2: 0.22365111111111113
Calculating scores for Sentiment 3...
Score for Supplier3: 0.2042
Processing category: Cost
Calculating scores for Cost 1...
Score for Quotation2.txt: 1.5
Calculating scores for Cost 2...
Score for Quotation1.txt: 1.0
Calculating scores for Cost 3...
Score for Quotation3.txt: 0.5
Processing category: Delivery
Calculating scores for Delivery 1...
Score for Quotation2.txt: 0.6000000000000001
Calculating scores